# TASK 1: SHAPE DETECTION AND CENTROID COORDINATES<BR>


**DESCRIPTION:** I was provided with an image, which I had to download on my system. The task was to detect the polygons present in the image and identify their centroids using the features of CONTOUR function and then I had to print the name of the polygon and the centroid coordinates of the polygon in a file using FILE HANDLING techniques. The submission includes both the file generated as well as the code.

In [1]:
import cv2 as cv
import numpy as np

## Stack Images Code Snippet (Taken from github)

In [2]:
def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv.cvtColor( imgArray[x][y], cv.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv.cvtColor(imgArray[x], cv.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver

## Creating function to compute number of sides/corners, shape & its centroid 

In [5]:
def getContours(img):
    contours, _ = cv.findContours(img, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE) 
    #Retrieval method = external (extreme outer contours, good for outer edges),
    #chain_approx = requests for all of the information or for compressed values of contours
    
    #save file in txt format
    with open("shapes.txt", "w") as file:
        for cnt in contours:
            area = cv.contourArea(cnt)
            print(f"Area: {area}")


            #give a min threshold for area, so that it does not detect any noise
            if area > 500:
                cv.drawContours(imgContour, cnt, -1, (0,255,0), 5)
                #calculate curve length to help us approx corners of our shape
                perimeter = cv.arcLength(cnt, True) # curve, closed = True
                print(f"Perimeter: {perimeter}")

                #approximate corner points:
                epsilon = 0.01 * perimeter #resolution
                approx = cv.approxPolyDP(cnt, epsilon, closed = True) #approx will give us the corner points
    #             print(approx)
                print(f"Number of sides: {len(approx)}")
                totalCorners = len(approx)

                #creating bounding/rect box around our object (x,y,w,h)
                x, y, w, h = cv.boundingRect(approx)


                # Calculating centroid of each shape (weighted avg of all pixels in blob/shape of similar intensities)
                M = cv.moments(approx)
                if M["m00"] != 0:
                    x_centroid = int(M["m10"] / M["m00"])
                    y_centroid = int(M["m01"] / M["m00"])
                    print(f"Centroid: ({x_centroid}, {y_centroid})")
                        
                    cv.circle(imgContour, (x_centroid, y_centroid), 5, (255, 0, 0), -1)

                #categorization into various shapes: 
                if totalCorners == 3: 
                    cv.putText(imgContour, "Triangle", (x_centroid, y_centroid), cv.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
                    file.write(f"Shape: Triangle\n")

                elif totalCorners == 4: 
                    #differentiate between rectangle and square:
                    #Square has aspect ratio between width and height as 1 (since width = height)

                    aspRatio = w/float(h) #since we are dealing with decimal numbers we define one of them as float

                    #deviation of 5% allowed:

                    if aspRatio > 0.95 and aspRatio < 1.05:
                        cv.putText(imgContour, "Square", (x_centroid, y_centroid), cv.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
                        file.write(f"Shape: Square\n")
                    else:
                        cv.putText(imgContour, "Rectangle", (x_centroid, y_centroid), cv.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
                        file.write(f"Shape: Rectangle\n")

                elif totalCorners == 5: 
                    cv.putText(imgContour, "Pentagon", (x_centroid, y_centroid), cv.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
                    file.write(f"Shape: Pentagon\n")

                elif totalCorners == 6: 
                    cv.putText(imgContour, "Hexagon", (x_centroid, y_centroid), cv.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
                    file.write(f"Shape: Hexagon\n")

                else:
                    cv.putText(imgContour, "Circle", (x_centroid, y_centroid), cv.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
                    file.write(f"Shape: Circle\n")
                    
            
                file.write(f"Area: {area}\n")
                file.write(f"Perimeter: {perimeter}\n")
                file.write(f"Centroid: ({x_centroid}, {y_centroid})\n\n")



In [6]:
path = ('C:\\Users\\iamsa\\Desktop\\CODES\\PYTHON\\OPEN CV\\OPENCV Crash Course\\Projects\\Task 1 - Shape Detection\\image.jpeg')
img = cv.imread(path)
imgContour = img.copy()

#### Step 1: Convert BGR Image to GrayScale <br>

(Compulsory step in case of colored images, but since we already have a b/w image it will visually not make much difference)

In [7]:
gray = cv.cvtColor(img, cv.COLOR_BGR2HSV)

#### Step 2: Apply Gausian Blur 

In [8]:
blur = cv.GaussianBlur(gray, (7,7), 1) #kernel value, sigma(x) --> measure of level of blur

#### Step 3: Applying Canny (for Edge Detection) on GrayScale Image <br>

Since finding contours (next step) works better on binary canny image or binary thresholded image

In [9]:
canny = cv.Canny(blur, 50, 150)

#### Step 4: Make predictions on our initial image by calling user defined function above

In [10]:
getContours(canny)

Area: 15992.0
Perimeter: 504.48528122901917
Number of sides: 4
Centroid: (89, 432)
Area: 23905.0
Perimeter: 771.8721451759338
Number of sides: 3
Centroid: (457, 474)
Area: 47434.0
Perimeter: 856.3818116188049
Number of sides: 6
Centroid: (145, 151)


In [11]:
imgBlank = np.zeros_like(img)
imgTitle = imgBlank.copy()
cv.putText(imgTitle, "Task 1 : Shape & Centroid", (60,300), cv.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
cv.putText(imgTitle, "Detection By Sanjana Dutta", (60,350), cv.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
imgStack = stackImages(0.6, ([img, gray, blur], [canny, imgContour, imgTitle]))

In [12]:
cv.imshow("Stack", imgStack)
# cv.imshow('task_pic', img)
cv.waitKey(0)
cv.destroyAllWindows()